<a href="https://colab.research.google.com/github/sagarikajadon13/Grammar-Error-Detection/blob/main/1_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import string

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import sent_tokenize

import spacy
import gensim
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, precision_score, recall_score, precision_recall_curve

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.pipeline import Pipeline

import xgboost as xgb

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
train= pd.read_csv('/content/drive/MyDrive/SHL NLP task/train_processed.csv')
val= pd.read_csv('/content/drive/MyDrive/SHL NLP task/val_data.csv')
test= pd.read_csv('/content/drive/MyDrive/SHL NLP task/test_data.csv')
test_inputs= test['input']

In [ ]:
train

,input,labels,input_len,preprocessed
0,I am reading score of Mahler is Symphony No . .,0,11,I am reading score of Mahler is Symphony No . .
1,I am not interested in cars or electric applia...,1,10,I am not interested in car or electric applian...
2,This is my homework for my English class .,0,9,This is my homework for my English class .
3,"In comparison , Canada is catches increased an...",0,18,"In comparison , Canada is catch increased and ..."
4,"Fortunately , my older sister is friend is a d...",1,11,"Fortunately , my older sister is friend is a d..."
...,...,...,...,...
19930,To read all the formal written meeting records...,1,25,To read all the formal written meeting record ...
19931,To check whether the written notices comply wi...,1,15,To check whether the written notice comply wit...
19932,To check the rules of procedure and reports of...,1,13,To check the rule of procedure and report of t...
19933,To check whether the supervisors are in charge...,1,18,To check whether the supervisor are in charge ...


In [ ]:
X_train= train['input']
y_train= train['labels']

X_val= val['input']
y_val= val['labels']

test= test['input']

In [ ]:
pipe0= Pipeline([('tfidf', TfidfVectorizer(lowercase= False, tokenizer= sent_tokenize)),
                # ('scaler', MaxAbsScaler()),
               ('model', MultinomialNB())])

In [ ]:
pipe0.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(lowercase=False,
                                 tokenizer=<function sent_tokenize at 0x7f4a790eee60>)),
                ('model', MultinomialNB())])

In [ ]:
y_pred= pipe0.predict(X_val)
f1_score(y_val, y_pred)

0.010964205095130602

In [ ]:
pipe1= Pipeline([('tfidf', TfidfVectorizer(lowercase= False, tokenizer= sent_tokenize)),
                ('scaler', MaxAbsScaler()),
               ('model', KNeighborsClassifier())])

In [ ]:
pipe1.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(lowercase=False,
                                 tokenizer=<function sent_tokenize at 0x7f4a790eee60>)),
                ('scaler', MaxAbsScaler()), ('model', KNeighborsClassifier())])

In [ ]:
y_pred= pipe1.predict(X_val)
f1_score(y_val, y_pred)

0.6482084690553745

In [ ]:
pipe2= Pipeline([('tfidf', TfidfVectorizer(lowercase= False, tokenizer= sent_tokenize)),
                ('scaler', MaxAbsScaler()),
               ('model', SVC())])

In [ ]:
pipe2.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(lowercase=False,
                                 tokenizer=<function sent_tokenize at 0x7f4a790eee60>)),
                ('scaler', MaxAbsScaler()), ('model', SVC())])

In [ ]:
y_pred= pipe2.predict(X_val)
f1_score(y_val, y_pred)

0.014805278403604762

In [ ]:
test_preds_knn= pipe1.predict(test)

In [ ]:
test_preds_knn

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
sub= pd.DataFrame({'input': test_inputs,
                   'labels': test_preds_knn})

In [ ]:
sub

,input,labels
0,I believe they will master Japanese soon becau...,1
1,I am looking for it .,1
2,Apple is a round fruit with smooth and colorfu...,1
3,Let It Will Be Push .,1
4,I rode on this ship from Sendai .,1
...,...,...
9967,Lang is growing up and improving step by step !,1
9968,I was away from my housework for about days .,1
9969,The conflict is so serious .,1
9970,An apple is a round fruit with smooth and colo...,1


In [ ]:
sub.to_csv('submission_knn.csv', index= False)